- Se importan las librerias con las que se va a trabajar

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from unidecode import unidecode
import time
import base64
import urllib.parse
import urllib.request
import json
import copy

- Primero se acceder a la página de banco provincia para buscar la dirección de la api en los contenidos que carga la página.
- Una vez que se identificó se copia el el codigo como "curl (as bash)" y se lo convierte para interpretarlo en python y de esta manera poder trabajarlo en Jupyer

In [2]:
headers = {
    'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
    'accept': '*/*',
    'content-type': 'application/json',
    'Referer': 'https://www.provinciacompras.com.ar/tecnologia',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    'workspace': 'master',
    'maxAge': 'medium',
    'appsEtag': 'remove',
    'domain': 'store',
    'locale': 'es-AR',
    '__bindingId': '7780ff79-9028-4a9e-8025-3bec4cd24948',
    'operationName': 'Categories',
    'variables': '{}',
    'extensions': '{"persistedQuery":{"version":1,"sha256Hash":"944799ff7758ce9487059bfcf79ecd99c26b4343e04489ad0647678ba77e7a10","sender":"baproar.components@0.x","provider":"vtex.store-graphql@2.x"}}',
}

response = requests.get('https://www.provinciacompras.com.ar/_v/segment/graphql/v1', params=params, headers=headers)

- Se guarda la respuesta como un json para trabajar.

In [3]:
data = response.json()["data"]

- El json viene como un diccionario que contiene listas y más diccionarios dentro para trabjar.
- Los productos están dividos por categorias que contienen subcategorias. El objetivo es conseguir el url de cada categoria particular para poder guardarlos.
- Se crea una lista con las categorias principales y luego una lista con las subcategorias.
- Si las subcategorias no tienen más categorias particulares se extrae directamente el href para crear la url y agregarlo a la lista de urls finalizados

In [4]:
categorias = data["categories"]
liCategoriasPrincipal = []
liSubcategoria = []
urlsFinalizado = []
def dividirCategorias(categoriaPrincipal):
    for categoria in categoriaPrincipal:
        if(categoria["children"]):
            dictCategorias = {categoria["name"]: categoria["children"]}
            liCategoriasPrincipal.append(dictCategorias)
            for subCategoria in categoria["children"]:
                if(subCategoria["children"]):
                    dictSubcategoria = {subCategoria["name"]: subCategoria["children"]}
                    liSubcategoria.append(dictSubcategoria)
                else:
                    linkSubcategoria = f"https://www.provinciacompras.com.ar{subCategoria['href']}"
                    urlsFinalizado.append(linkSubcategoria)

In [5]:
dividirCategorias(categorias)

- Se realiza el mismo procedimiento anterior para la lista subcategorias. Se divide el procedimiento en otro bucle for para que se lea más fácil

In [6]:
def dividirSubcategorias(subcategoria):
    for cat in subcategoria:
        #La respuesta de cada categoría viene en un diccionario, por lo que se realiza otro bucle for para recorrerlo.
        for nameSubCategoria in cat:
            #Cada respuesta viene en una lista, se la recorre para extraer el 'href' correspondiente
            liLinksSubCategoria = [x["href"] for x in cat[nameSubCategoria]]
            #Con las lista de de 'href' obtenidos se agrega cada una a la lista ya existente de "urlsFinalizados"
            for linkSubCategoria in liLinksSubCategoria:
                urlsFinalizado.append(f"https://www.provinciacompras.com.ar{linkSubCategoria}")

In [7]:
dividirSubcategorias(liSubcategoria)

- Se ordena la lista de urls alfabeticamente

In [8]:
urlsFinalizado.sort()

    Analizando la API de ProvCompras dentro de los parámetros para realizar la query tiene un valor que se integra en el diccionario de la siguiente manera : {..., 'query': 'tecnologia/celulares-y-tablets/accesorios',...}. Por lo que se procede a extraer cada parte especifica de la url para obtener cada consulta de las categorias
    Luego se guardan en una lista para usar en el futuro

In [9]:
urlPrincipal = 'https://www.provinciacompras.com.ar/'
liQueryCategoria = [url.replace(urlPrincipal, "") for url in urlsFinalizado]

    Por una cuestión unicamente de orden se crea un diccionario con las categorias principales y que cada categoría contenga sus url, no es algo estrictamente necesario para realizar el procedimiento de extracción

In [10]:
dictQueryCategorias = {}
for query in liQueryCategoria:
    categoriaPrincipal = query.split("/")[0]
    if categoriaPrincipal in dictQueryCategorias:
        dictQueryCategorias[categoriaPrincipal].append(query)
    else:
        dictQueryCategorias[categoriaPrincipal] = [query]

- Luego de obtener todas las querys y las categorias se realiza un procedimientos para obtener las paquetes de los productos.
- Primero se accede a la url de la API del primer link de la lista de urls de categorias.
- Una vez que se accede se procede a desarmar la api para ver como son los requisitos de parametros que utiliza para poder realizar las consultas
- Cuando se identifican los parámetros se relizan los distintos cambios para cada url y que la consulta no falle

In [11]:
urlInicial = "https://www.provinciacompras.com.ar/_v/segment/graphql/v1?workspace=master&maxAge=short&appsEtag=remove&domain=store&locale=es-AR&__bindingId=7780ff79-9028-4a9e-8025-3bec4cd24948&operationName=productSearchV3&variables=%7B%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2240b843ca1f7934d20d05d334916220a0c2cae3833d9f17bcb79cdd2185adceac%22%2C%22sender%22%3A%22vtex.store-resources%400.x%22%2C%22provider%22%3A%22vtex.search-graphql%400.x%22%7D%2C%22variables%22%3A%22eyJoaWRlVW5hdmFpbGFibGVJdGVtcyI6dHJ1ZSwic2t1c0ZpbHRlciI6IkFMTCIsInNpbXVsYXRpb25CZWhhdmlvciI6ImRlZmF1bHQiLCJpbnN0YWxsbWVudENyaXRlcmlhIjoiTUFYX1dJVEhPVVRfSU5URVJFU1QiLCJwcm9kdWN0T3JpZ2luVnRleCI6ZmFsc2UsIm1hcCI6ImMsYyxjIiwicXVlcnkiOiJjYWx6YWRvL2ZlbWVuaW5vL2JvdGluZXMiLCJvcmRlckJ5IjoiT3JkZXJCeVNjb3JlREVTQyIsImZyb20iOjAsInRvIjoxNywic2VsZWN0ZWRGYWNldHMiOlt7ImtleSI6ImMiLCJ2YWx1ZSI6ImNhbHphZG8ifSx7ImtleSI6ImMiLCJ2YWx1ZSI6ImZlbWVuaW5vIn0seyJrZXkiOiJjIiwidmFsdWUiOiJib3RpbmVzIn1dLCJmYWNldHNCZWhhdmlvciI6IkR5bmFtaWMiLCJjYXRlZ29yeVRyZWVCZWhhdmlvciI6ImRlZmF1bHQiLCJ3aXRoRmFjZXRzIjpmYWxzZSwidmFyaWFudCI6IiJ9%22%7D"

In [12]:
#Se dividide en partes la url

partesUrl = urllib.parse.urlparse(urlInicial)
print(partesUrl)
print("---")

#Una de las partes es un parámetro "quey". Se obtienenen la query para conocer los parámetros necesarias para pedir el request a la API
query = urllib.parse.parse_qs(partesUrl.query)
print(query)
print("----")

#Se interpretan las querys para ver como son los pedidos a la API. Las query tienen un parámetro "extensions"
#que dentro contiene las variables para realizar los pedidos correspondientes a la API para cada categoria
extensions = json.loads(query['extensions'][0])

#Una vez obtenido 'extensions' se accede a su parámetro 'variables' que es lo que hay que modificar para cada pedido de la API según la categoria 
variables = extensions['variables']
print(variables)
print("------")

#Las variables vienen codificadas por lo que hay decodificarlas para que las podamos leer e interpretar.
variablesDecodificadas = base64.b64decode(variables.encode()).decode()
variablesDecodificadas = json.loads(variablesDecodificadas)
print('--- replace values ---')
print(variablesDecodificadas)

ParseResult(scheme='https', netloc='www.provinciacompras.com.ar', path='/_v/segment/graphql/v1', params='', query='workspace=master&maxAge=short&appsEtag=remove&domain=store&locale=es-AR&__bindingId=7780ff79-9028-4a9e-8025-3bec4cd24948&operationName=productSearchV3&variables=%7B%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2240b843ca1f7934d20d05d334916220a0c2cae3833d9f17bcb79cdd2185adceac%22%2C%22sender%22%3A%22vtex.store-resources%400.x%22%2C%22provider%22%3A%22vtex.search-graphql%400.x%22%7D%2C%22variables%22%3A%22eyJoaWRlVW5hdmFpbGFibGVJdGVtcyI6dHJ1ZSwic2t1c0ZpbHRlciI6IkFMTCIsInNpbXVsYXRpb25CZWhhdmlvciI6ImRlZmF1bHQiLCJpbnN0YWxsbWVudENyaXRlcmlhIjoiTUFYX1dJVEhPVVRfSU5URVJFU1QiLCJwcm9kdWN0T3JpZ2luVnRleCI6ZmFsc2UsIm1hcCI6ImMsYyxjIiwicXVlcnkiOiJjYWx6YWRvL2ZlbWVuaW5vL2JvdGluZXMiLCJvcmRlckJ5IjoiT3JkZXJCeVNjb3JlREVTQyIsImZyb20iOjAsInRvIjoxNywic2VsZWN0ZWRGYWNldHMiOlt7ImtleSI6ImMiLCJ2YWx1ZSI6ImNhbHphZG8ifSx7ImtleSI6ImMiLCJ2YWx1ZSI6ImZlbWVuaW5vIn0seyJrZXki

- De las variables se observa que:
  - Hay una clave 'query' que hay que modificar con las categorias.
  - La respuesta trae de en principio 18 elementos que van desde lo que se especifica en "from" hasta "to".
  - Dentro de la clave "selectedFacets", que es una lista de diccionarios, hay que cambiar las claves 'value' para cada categoría
  - Primero hay que cambiar la calve 'query' y las claves de 'value' dentro de 'selectedFacets' para poder conseguir de forma correcta cada URL que haga el pedido a la API. Una vez que se realizó eso luego se puede obtener la respuesta, para analizar cuantos productos tiene cada categoria en total para poder modificar los elementos a traer. 
  - Hay que realizar el proceso inverso de desarmar la url para ir conviertiendo cada elemento según la categoria, y de esta manera obtener cada url modificada.

In [14]:
#Como es un procedimiento largo, se divide en varios bloques para ir analizando cada paso.
#Se crea una lista para guardar cada variable decodificada correspondiente de la lista original.
liVariablesDecodificadas = []

#Hay que dividir las categorias para obtener los 'value' y luego reemplazarlos en las variables.
for link in liQueryCategoria:
    #Se divide los valores de la url para tenerlos por separado
    valoresSeparado = link.split("/")

    #Se crea una copy de cada parametro de la query para que cada copia sea independiente y de esta manera no se pisen.
    #Esto se realiza porque como es un elemento string, si se pisan terminan apuntando al mismo elemento de memoria y se modifican todos.
    #Se usa deepcopy porque esto permite crear una copia de cada elemento anidado dentro de la variable que estamos copiando
    copiaParametrosVariables = copy.deepcopy(variablesDecodificadas)
    
    #Se modifican los valores
    #Primero se modifica la clave 'query' para agregar cada correspondiente categoria.
    copiaParametrosVariables["query"] = link

    #Se modifican las clave 'value'. Esto se realiza en una lista para que se itere sobre cada elemento. Se agrega una condición if para evitar que se vaya de rango.
    for indice, valor in enumerate(copiaParametrosVariables["selectedFacets"]):
        if indice < len(valoresSeparado):
            valor["value"] = valoresSeparado[indice]
    #Se agregan todas las variables decoficadas a la lista. Con esa lista se procede a trabajar en siguiente paso
    liVariablesDecodificadas.append(copiaParametrosVariables)

Se recrean las copias de cada string obtenidos en el primer para poder obtener las url que nos redirecciones a cada query

In [15]:
#En este paso se procede a codificar las variables. Y se las agrega a una nueva lista.
liVariablesCodificadas = []
for variableDecodificada in liVariablesDecodificadas:
    #Se codifica para variable decodificada.
    variableDecodificada = json.dumps(variableDecodificada)

    #Se crea una copia las variables codificadas.
    copiaVariables = copy.deepcopy(variables)

    #Se reemplaza el valor con las variables codificadas que se obtuvieron.
    copiaVariables = base64.b64encode(variableDecodificada.encode()).decode()
    liVariablesCodificadas.append(copiaVariables)

In [16]:
#En este paso se agregan las variables ya codificadas al parámetro 'extensions'.
liExtensions = []
for varCodificada in liVariablesCodificadas:
    #Se realiza una copia de la variable 'extensions'
    copiaExtensions = copy.deepcopy(extensions)

    #Se modifican los valores en 'extensions'
    copiaExtensions['variables'] = varCodificada
    liExtensions.append(copiaExtensions)

In [17]:
liQuerys = []
for extension in liExtensions:
    copiaQuery = copy.deepcopy(query)
    copiaQuery['extensions'][0] = json.dumps(extension)
    liQuerys.append(copiaQuery)

In [22]:
liPartes = []
for queryModificada in liQuerys:
    copiaPartes = copy.deepcopy(partesUrl)
    copiaPartes = copiaPartes._replace(query=urllib.parse.urlencode(queryModificada, doseq=True))
    liPartes.append(copiaPartes)

In [23]:
urlLista = []
for parteModifica in liPartes:
    copiaUrl = copy.deepcopy(urlInicial)
    copiaUrl = urllib.parse.urlunparse(parteModifica)
    urlLista.append(copiaUrl)

- Una vez obtenidas las URL de cada categoria. Se buscan los productos.
- Los productos se filtran según la cantidad de productos que existan en la categoria. Si la categoría tiene más de 18 productos se van a tener que realizar dos requests distintos porque en cada respuesta va a traer 18 productos como maximo.
- Para saber cuando productos tiene la categoria hay hacer un GET a la url. Y capturar del diccionario  {productSearch: {recordsFiltered: X}}. El valor de esa variable es la cantidad de productos que hay.
 - Se crean dos listas:
   - Una lista contiene los requests de las categorias que tienen menos de 18 productos, por lo que con esas respuestas ya se puede extraer la información necesaria
   - Una lista que contiene los requests de las categorías que tienen más de 18 productos. Esta lista hay que seguir trabajandola para poder obtener la información necesaria

In [31]:
productosFinalizados = []
productosSinFinalizar = []
userAgent = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
#La función se corre con un time.sleep(2) porque con menos fallaba al traer las consultas.
for productos in urlLista:
    info = requests.get(productos, headers=userAgent)
    infoJson = info.json()
    cantidadProductos = infoJson["data"]["productSearch"]["recordsFiltered"]
    if cantidadProductos > 0 and cantidadProductos < 19:
        productosFinalizados.append(infoJson)
    elif cantidadProductos >= 19:
        productosSinFinalizar.append(productos)
    time.sleep(2)

In [32]:
with open('productosSinFinalizarUrl.json', 'w') as file:
    json.dump(productosSinFinalizar, file)

In [33]:
listaProductosFinalizados = []
for prod in productosFinalizados:
    prodFinalizado = prod["data"]["productSearch"]["products"]
    listaProductosFinalizados.append(prodFinalizado)

In [34]:
resultados = []
for prub in listaProductosFinalizados:
    for pr in prub:
        nombre = pr['productName']
        precio = pr['priceRange']['sellingPrice']['highPrice']
        categoria = pr["categories"][0]
        link = f'https://www.provinciacompras.com.ar{pr["link"]}'
        productoDict = {"Producto": nombre,
                    "Precio": precio,
                    "Link": link,
                    "Categoria":categoria}
        resultados.append(productoDict)

In [35]:
resultadosLimpio =[]
resultadosLimpio.extend(resultados)

In [36]:
resultadosDF = pd.DataFrame(resultadosLimpio)

In [37]:
type(resultadosDF)

pandas.core.frame.DataFrame

In [38]:
resultadosDF.to_excel("ProvCompras.xlsx",index=False)